<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/ParticleSimulator3DSimple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import numpy as np
import plotly.graph_objects as go

class Simulator:
    def __init__(self, world_size):
        self.world_size = world_size
        self.positions = np.empty((0, 3), dtype=np.float32)
        self.velocities = np.empty((0, 3), dtype=np.float32)
        self.colors = np.empty(0, dtype=object)
        self.radius = 1.5
        self.radius_sq = self.radius ** 2

    def add_particle(self, position, velocity, color):
        self.positions = np.vstack([self.positions, position])
        self.velocities = np.vstack([self.velocities, velocity])
        self.colors = np.concatenate([self.colors, [color]])

    def check_collisions(self):
        if len(self.positions) < 2:
            return

        for i in range(len(self.positions)):
            for j in range(i + 1, len(self.positions)):
                dist_sq = np.sum((self.positions[i] - self.positions[j]) ** 2)
                if dist_sq < self.radius_sq:
                    self.velocities[i], self.velocities[j] = self.velocities[j], self.velocities[i]

    def step(self):
        if len(self.positions) > 0:
            self.positions += self.velocities
            self.check_collisions()

def animate(sim, frames):
    n_particles = len(sim.positions)
    history_pos = np.empty((frames, n_particles, 3), dtype=np.float32)

    for frame_idx in range(frames):
        history_pos[frame_idx] = sim.positions.copy()
        sim.step()

    fig = go.Figure()

    colors = ['yellow', 'green', 'blue']
    for i, color in enumerate(colors):
        fig.add_trace(go.Scatter3d(
            x=[history_pos[0, i, 0]],
            y=[history_pos[0, i, 1]],
            z=[history_pos[0, i, 2]],
            mode='markers',
            marker=dict(size=10, color=color),
            name=f'Частица {i+1} ({color})'
        ))

    frames_list = []
    for frame_idx in range(frames):
        frame_data = []
        for i, color in enumerate(colors):
            frame_data.append(go.Scatter3d(
                x=[history_pos[frame_idx, i, 0]],
                y=[history_pos[frame_idx, i, 1]],
                z=[history_pos[frame_idx, i, 2]],
                mode='markers',
                marker=dict(size=10, color=color),
                showlegend=False
            ))

        frames_list.append(go.Frame(
            data=frame_data,
            name=str(frame_idx)
        ))

    fig.frames = frames_list

    fig.update_layout(
        scene=dict(
            xaxis=dict(range=[0, sim.world_size]),
            yaxis=dict(range=[0, sim.world_size]),
            zaxis=dict(range=[0, sim.world_size]),
            aspectmode='cube'
        ),
        updatemenus=[
            dict(
                type='buttons',
                showactive=False,
                buttons=[
                    dict(
                        label='▶️',
                        method='animate',
                        args=[None, dict(
                            frame=dict(duration=50, redraw=True),
                            fromcurrent=True,
                            mode='immediate'
                        )]
                    ),
                    dict(
                        label='⏸️',
                        method='animate',
                        args=[[None], dict(
                            frame=dict(duration=0, redraw=False),
                            mode='immediate'
                        )]
                    ),
                ]
            ),
        ]
    )

    return fig


sim = Simulator(20)

position1 = np.array([18.0, 10.0, 10.0], dtype=np.float32)
velocity1 = np.array([-0.8, 0.0, 0.0], dtype=np.float32)

position2 = np.array([16.93, 13.46, 10.0], dtype=np.float32)
velocity2 = np.array([-0.693, -0.4, 0.0], dtype=np.float32)

position3 = np.array([16.93, 6.54, 10.0], dtype=np.float32)
velocity3 = np.array([-0.693, 0.4, 0.0], dtype=np.float32)

sim.add_particle(position1, velocity1, 'yellow')
sim.add_particle(position2, velocity2, 'green')
sim.add_particle(position3, velocity3, 'blue')

fig = animate(sim, frames=20)
fig.show()